1. Descripción de la arquitectura apoyándose en el artículo original.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from keras.applications import MobileNet
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

#importamos las librerías necesarias
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import keras
import tensorflow as tf
from keras.applications.imagenet_utils  import preprocess_input, decode_predictions
import numpy as np

from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input

In [4]:
# Ruta al directorio con los datos
fpath = './Imagenes_para_exposiciones_Grupo_2' 
random_seed = 82  # Semilla para reproducibilidad
np.random.seed(random_seed)

In [5]:
# Obtener todos los archivos de imagen
all_images = [f for f in os.listdir(fpath) if f.lower().endswith(('.jpg'))]
categories = set(img.split('_')[0] for img in all_images)  # Extraer categorías del nombre del archivo
print("Lista de categorias = ",categories,"\nNro de categorias = ", len(categories))

Lista de categorias =  {'aptenodytes-forsteri', 'chrysemys-picta', 'aethia-cristatella', 'bison-bison', 'agalychnis-callidryas', 'apis-mellifera', 'bradypus-variegatus', 'betta-splendens', 'danaus-plexippus', 'ailurus-fulgens', 'crotalus-atrox', 'correlophus-ciliatus', 'dasypus-novemcinctus', 'chelonia-mydas', 'circus-hudsonius', 'branta-canadensis', 'crocodylus-niloticus', 'cryptoprocta-ferox', 'balaenoptera-musculus', 'dendrobatidae', 'ardea-herodias', 'acinonyx-jubatus', 'architeuthis-dux', 'centrochelys-sulcata', 'canis-lupus-familiaris', 'codium-fragile', 'canis-lupus', 'ceratotherium-simum', 'anas-platyrhynchos', 'centruroides-vittatus', 'crotophaga-sulcirostris', 'cathartes-aura', 'bos-taurus', 'coelacanthiformes', 'connochaetes-gnou', 'dermochelys-coriacea', 'desmodus-rotundus', 'agkistrodon-contortrix', 'alces-alces', 'ceratitis-capitata', 'ailuropoda-melanoleuca', 'carcharodon-carcharias', 'aquila-chrysaetos', 'cyanocitta-cristata'} 
Nro de categorias =  44


In [6]:
# Crear mapeo de categorías a índices
category_to_index = {cat: idx for idx, cat in enumerate(categories)}

In [7]:
# Cargar imágenes y etiquetas
def load_images_and_labels(image_files):
    images = []
    labels = []
    for img_name in image_files:
        img_path = os.path.join(fpath, img_name)
        img = image.load_img(fpath+"/"+img_name, target_size=(224,224,3)) #carga la imagen y ajusta su tamaño
        img = img_to_array(img) / 255  # Normalizar los píxeles
        images.append(img)
        label = category_to_index[img_name.split('_')[0]]
        labels.append(label)
    return np.array(images), np.array(labels)

In [8]:
# Análisis inicial de la distribución de clases
image_count_per_category = {cat: 0 for cat in categories}
for img in all_images:
    category = img.split('_')[0]
    image_count_per_category[category] += 1

In [9]:
# Mostrar el número de imágenes por categoría
print("Número de imágenes por categoría:", image_count_per_category)

Número de imágenes por categoría: {'aptenodytes-forsteri': 1, 'chrysemys-picta': 1, 'aethia-cristatella': 3, 'bison-bison': 1, 'agalychnis-callidryas': 3, 'apis-mellifera': 4, 'bradypus-variegatus': 2, 'betta-splendens': 2, 'danaus-plexippus': 1, 'ailurus-fulgens': 2, 'crotalus-atrox': 1, 'correlophus-ciliatus': 1, 'dasypus-novemcinctus': 1, 'chelonia-mydas': 1, 'circus-hudsonius': 3, 'branta-canadensis': 5, 'crocodylus-niloticus': 2, 'cryptoprocta-ferox': 2, 'balaenoptera-musculus': 2, 'dendrobatidae': 4, 'ardea-herodias': 3, 'acinonyx-jubatus': 2, 'architeuthis-dux': 2, 'centrochelys-sulcata': 4, 'canis-lupus-familiaris': 2, 'codium-fragile': 4, 'canis-lupus': 2, 'ceratotherium-simum': 2, 'anas-platyrhynchos': 4, 'centruroides-vittatus': 3, 'crotophaga-sulcirostris': 5, 'cathartes-aura': 2, 'bos-taurus': 1, 'coelacanthiformes': 1, 'connochaetes-gnou': 1, 'dermochelys-coriacea': 1, 'desmodus-rotundus': 4, 'agkistrodon-contortrix': 3, 'alces-alces': 2, 'ceratitis-capitata': 2, 'ailurop

2. Implementación (incluyendo argumentos para iniciar el modelo con Keras)

In [10]:
modelo_mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) #Modelo sin las capas densas (top)

#capas densas (totalmente conectadas) en la parte superior

#Capa de agrupación global y capa densa (totalmente conectada) en la parte superior del modelo
x = modelo_mobilenet.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Por ejemplo, una capa densa con 1024 neuronas
predictions = Dense(len(categories), activation='softmax')(x)  # Cambia "num_classes" al número de clases de tu problema

modelo_final = Model(inputs=modelo_mobilenet.input, outputs=predictions)

modelo_final.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

modelo_final.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

3. Aplicación del modelo en el conjunto de imágenes asignadas al grupo.
Discutir los resultados mostrando las clases presentes en sus datos.